In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 22 17:12:53 2018

@author: Medcare,Wan
"""
# score 58.80403
# rank 19+
import numpy as np
import xgboost as xgb
import pandas as pd

from sklearn.preprocessing import StandardScaler,MinMaxScaler
path='C:/Users/wwwkd/Desktop/competition/data'
pre_data=pd.read_csv(path+'/data_pre.csv')
test_data=pd.read_csv(path+'/tap_fun_test.csv')
train_data=pd.read_csv(path+'/tap_fun_train.csv')

#------------------------------------------------
import datetime
import numpy
# 国内公测时间 2017年8月13日
# 增加 test_data 的 注册日期距离公测时间的天数
def add_time_dif(array):
    new_array = numpy.zeros(shape=(array.shape[0],1))

    for i in np.arange(array['register_time'].shape[0]):
        the_array = array['register_time']
        sd = datetime.datetime.strptime('2017-08-13 00:00:00', '%Y-%m-%d %H:%M:%S')
        d = datetime.datetime.strptime(the_array[i], '%Y-%m-%d %H:%M:%S')
        time = (d - sd).days + 1
        new_array[i] = int(time)    
    array['time_dif'] = new_array
    #修改为int16 减少计算量
    array['time_dif'] = numpy.int16(array['time_dif'])
    return array
    # print (test_data['time_dif'])
test_data = add_time_dif(test_data)

# 增加 train_data 的 注册日期距离公测时间的天数
def my_none():
    new_array = numpy.zeros(shape=(train_data.shape[0],1))
    array = np.array(train_data['register_time']).reshape(len(train_data['register_time']),1)
    for i in np.arange(train_data['register_time'].shape[0]):

        sd = datetime.datetime.strptime('2017-08-13 00:00:00', '%Y-%m-%d %H:%M:%S')
        d = datetime.datetime.strptime(array[i,0], '%Y-%m-%d %H:%M:%S')
        time = (d - sd).days + 1
        new_array[i] = int(time)    
    train_data['time_dif'] = new_array
    # #修改为int16 减少计算量
    train_data['time_dif'] = numpy.int16(train_data['time_dif'])
my_none()

#-----------------------------------------------------------



train_data.drop_duplicates( keep='first', inplace=True)
del train_data['register_time'],test_data['register_time']
pre_data.drop_duplicates( keep='first', inplace=True)
train_data=train_data[train_data['pay_price']>0]
test_data_prect=test_data[test_data['pay_price']>0]
test_data_rule=test_data[test_data['pay_price']==0]
'''
def data_clear(df):
    temp=pd.DataFrame(index=range(1))
    temp['user_id']=df['user_id'].unique()[0]
    clear_sum=0
    for i in pre_data['wood']:
        clear_sum=df[str(i)+'_reduce_value'].max()+clear_sum
    if clear_sum==0:
        temp['is_clear']=1
    else:
        temp['is_clear']=0
    return temp    
'''
def rate_feat(df):
    for i in pre_data['wood']:
        df[str(i)+'rate']=df[str(i)+'_reduce_value']/df[str(i)+'_add_value']        
    df['pvp_rate']=df['pvp_lanch_count']/df['pvp_battle_count']
    df['pvp_win_rate']=df['pvp_win_count']/df['pvp_battle_count']
    df['pay_count_rate']=df['pay_count']/df['avg_online_minutes']
    df['pay_mean']=df['pay_price']/df['pay_count']
    df['pve_win_rate']=df['pve_win_count']/df['pve_battle_count']
    df['battle_count']=df['pve_battle_count']+df['pvp_battle_count']
    df['pvp_battle_rate']=df['pve_battle_count']/df['battle_count']
    df['win_rate']=(df['pve_win_count']+df['pvp_win_count'])/df['pvp_battle_rate']
    df['pve_rate']=df['pve_lanch_count']/df['pve_battle_count']
    df['lanch_count']=df['pvp_lanch_count']+df['pve_lanch_count']
    df['lanch_rate']=df['pvp_lanch_count']/df['lanch_count']
    df['battle_rate']=df['pvp_battle_count']/df['pve_battle_count']
    df['lauch_battle_rate']=df['pvp_lanch_count']/df['pve_lanch_count']
    df['treat_rate']=df['treatment_acceleration_reduce_value']/df['treatment_acceleraion_add_value']
    df['shaman_battle_rate']=df['shaman_reduce_value']/df['battle_count']    
    df['cavalry_battle_rate']=df['cavalry_reduce_value']/df['battle_count']
    df['infantry_battle_rate']=df['infantry_reduce_value']/df['battle_count']
    #df['infantry_loss_battle_rates']=(df['infantry_reduce_value']+df['wound_infantry_add_value'])/df['battle_count']
    #df['cavalry_loss_battle_rates']=(df['cavalry_reduce_value']+df['wound_cavalry_add_value'])/df['battle_count']
    #df['shaman_loss_battle_rates']=(df['shaman_reduce_value']+df['wound_shaman_add_value'])/df['battle_count']
    
    
    df['pvp_passive_rate']=(df['pvp_battle_count']-df['pve_lanch_count'])/df['time_dif']

    
    #df['all_reduce_count']=df['infantry_reduce_value']+df['cavalry_reduce_value']+df['shaman_reduce_value']
    #df['sodier_count']=df['infantry_add_value']+df['cavalry_add_value']+df['shaman_add_value']
    #df['infantry_add_rate']=df['infantry_add_value']/df['sodier_count']
    #df['cavalry_add_rate']=df['cavalry_add_value']/df['sodier_count']
    #df['shaman_add_rate']=df['shaman_add_value']/df['sodier_count']
    #df['wound_left_count']=df['wound_cavalry_reduce_value']+df['wound_infantry_reduce_value']+df['wound_shaman_reduce_value']
    #df['pvp_lanch_freque']=df['pvp_lanch_count']/df['avg_online_minutes']
    #df['pve_freque']=df['pve_lanch_count']/df['avg_online_minutes']
    #df['pvp_freque']=df['pvp_battle_count']/df['avg_online_minutes']
    df['wound_left_rate']=(df['wound_cavalry_reduce_value']+df['wound_infantry_reduce_value']+df['wound_shaman_reduce_value'])/df['treatment_acceleration_reduce_value']
    return df
#test_feat=test_data_prect
#train_feat=train_data
test_feat=rate_feat(test_data_prect)
train_feat=rate_feat(train_data)
print('clear_over')
#train_data=train_data[train_data['is_clear']==0]


#test_feat= test_data_prect[['user_id','pvp_battle_count','pvp_lanch_count','pay_price','avg_online_minutes']]
#train_feat= train_data[['user_id','pay_price','pvp_battle_count','pvp_lanch_count','prediction_pay_price','avg_online_minutes']]   
#feat=test_data.groupby('user_id').apply(lambda x: rate_feat(x))
print('test feat over')
#test_feat=test_feat.merge(feat,on='user_id',how='left')  
#feat=train_data.groupby('user_id').apply(lambda x: rate_feat(x))
print('train feat over')
#train_feat=train_feat.merge(feat,on='user_id',how='left')
#scaler = MinMaxScaler()
def data_scar(data):
    '''
    #data=data.reshape(-1, 1) 
    scaler.fit(data)
    data=scaler.fit_transform(data)
    '''
    data=(data-data.min())/(data.max()-data.min())
    return data
  
drop = ['user_id','pay_price']
train_feat['label']=train_feat['prediction_pay_price']/train_feat['pay_price']
train_idx=train_data[drop]
test_idx=test_data_prect[drop]
test_idx_rule=test_data_rule[drop]
y = train_feat.pop('label')
del train_feat['prediction_pay_price']
#test_feat['prediction_pay_price']=-111
data_feat=pd.concat([train_feat, test_feat])
data_feat=data_feat.drop(drop, axis=1)
#data_feat=data_feat.replace(np.Inf,0)
#data_feat=data_feat.replace(np.NaN,0)
#data_feat=data_feat.apply(lambda x:data_scar(x),axis = 0)
#data_feat=data_feat.replace(np.Inf,0)
y=y.replace(np.NaN,0)
train_feat=data_feat[:len(y)]
test_feat=data_feat[len(y):]

#test_feat=test_feat.drop('prediction_pay_price', axis=1)

#train_feat=train_feat.drop(drop, axis=1)
#test_feat=test_feat.drop(drop, axis=1)


#cols = ['pay_price']
'''
param = {'learning_rate' : 0.01,
        #'n_estimators': 1000,
        'max_depth': 6,
        'min_child_weight': 5,
        'gamma': 0,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'eta': 0.01,
        #'silent': 1,
        'objective':
        'reg',
        #'scale_pos_weight':1,
        'seed':1024}
dtrain = xgb.DMatrix(train_feat,label=y)

bst = xgb.train(param, dtrain, num_boost_round=100)
dtest = xgb.DMatrix(test_feat)
y_prob = bst.predict(dtest)
'''

'''
from sklearn.linear_model import LinearRegression
print('start')

lr = LinearRegression()
lr.fit(train_feat, y)

y_prob = lr.predict(test_feat)

'''

import lightgbm as lgb
lgb_train = lgb.Dataset(train_feat, y)
print('开始训练......')

params = {
            'boosting_type': 'gbdt',
            'metric': {'l2',}, 
#             'is_unbalance':'True',
            'learning_rate' : 0.01, 
             'verbose': 0,
#            'num_leaves': 32,
#             'max_depth':4, 
#             'max_bin':10, 
#             'lambda_l2': 10, 
            'objective': 'regression', 
            #'feature_fraction': 0.4,
            #'bagging_fraction':0.7, # 0.9是目前最优的
#             'bagging_freq':1,  # 3是目前最优的
#             'min_data': 500,
            'seed': 1024,
            'nthread': 12,
#             'silent': True,
}

'''
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'}
}
'''
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=170,#220
                #valid_sets=lgb_eval
                )

y_prob = gbm.predict(test_feat)

#y_prob=y_prob*test_feat['pay_price']
test_idx['label'] = y_prob
test_idx['prediction_pay_price']=test_idx['label']*test_idx['pay_price']
test_idx_rule['prediction_pay_price']=0

res = pd.concat([test_idx, test_idx_rule])

res[['user_id','prediction_pay_price']].to_csv("new_add_dif_time.csv", sep=',',index=False)



D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib

clear_over
test feat over
train feat over
开始训练......


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\